# 2 - 词袋模型
**词袋**(Bag-of-words)是将一个文档看作一个袋子，词就是袋子中的物品，不考虑其出现的先后顺序，只考察是否出现以及出现的频数。

而进一步，引入[tf-idf]()的概念，将每个文档中的词的频数替换为其tf-idf值，则可以选取文档中最具有代表性的词。

通过建立词袋模型，我们可以得到：
+ 所有词的**look-up table**（储存索引的字典），{'this':101,'is':3,'illogical':3421...}
+ 每个文档的**词袋表示**：doc1-> ((101,2), (3,5), (3421,2),...)
+ 用于计算每个词tf-idf值的**模型**
+ 每个文档的**tf-idf表示**：doc1-> ((101,0.4233), (3,0.5644), (3421,0.8119)...)

将文档用tf-idf词袋表示后，对其中的词进行**排序**，提取关键信息。

接下来使用[gensim](https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#sphx-glr-auto-examples-core-run-core-concepts-py)实现建模。

In [ ]:
import os
import numpy as np
import re

import spacy
from gensim import corpora
from gensim import models

### 构建模型

In [1]:
titles = []
docs = []
# 读取数据
for root,dirs,files in os.walk('raw_data/'):
    for file in files:
        if file.find('.txt')<0: continue
        with open(os.path.join(root,file)) as f:
            season = root.split('/')[-1]
            episode = re.search(r'^\d+',file).group(0)
            titles.append('{}_{}'.format(season,episode))
            docs.append(f.read())

In [2]:
# 数据预处理
nlp = spacy.load('en') #借助spcay

def preprocess(text):
    doc = nlp(text.lower())
    words = []
    for token in doc:
        # 去除符号、停词、长度为一的字符，并进行词形还原
        if token.pos_!='PUNCT' and not token.is_stop and len(set(token.text))>1:
            words.append(token.lemma_)
    return words

def prep_data(doc): 
    # 在分词前对原始台词数据进行初步处理
    doc = doc.split('\n')
    lines = ['']
    for l in doc:
        if l.find('[')>0 or l.find(']')>0: #去除场景说明
            continue
        
        if l.find(':')>0: #去除每句台词前的角色说明
            l = l[l.index(':')+2:]
            lines.append(l)
        else:
            lines[-1] += ' '+l #将一句台词连成一个句子
    
    return preprocess(' '.join(lines))


docs = list(map(prep_data,docs))
processed_corpus = np.array(docs).flatten()

开始构建**词袋模型**。

核心元素：
+ dictionary = `corpora.Dictionary`(processed_corpus)。将所有文档切分至词，一起用于构建字典
+ doc_bow = `dictionary.doc2bow`(doc)。输入分词后的文档得到其词袋表示
+ tfidf_model = `models.TfidfModel`(bow_corpus)。利用所有文档的词袋表示训练tf-idf模型
+ doc_tfidf = `tfidf[doc_bow]`。利用模型将词袋表示转化为tfidf表示

In [3]:
dictionary = corpora.Dictionary(processed_corpus)
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]
tfidf_model = models.TfidfModel(bow_corpus)

In [4]:
# 获取字典
stoi = dictionary.token2id
print("words num:",len(stoi))
print("\ndictionary:\n",list(stoi.items())[10:20]) 
#没有id2token方法，需要用zip(dict.values(),dict.keys())自定义
itos = dict(zip(stoi.values(),stoi.keys()))

words num: 11655

dictionary:
 [('achieve', 10), ('acknowledge', 11), ('activate', 12), ('actually', 13), ('adc', 14), ('additional', 15), ('address', 16), ('affectionate', 17), ('affirmative', 18), ('afraid', 19)]


### 提取feature words

In [6]:
# 得到词袋表示
doc_bow = dictionary.doc2bow(docs[0])
print("doc_bow:\n",doc_bow[10:20])

# 得到tf-idf表示
doc_tfidf = tfidf_model[doc_bow]
print("\ndoc_tfidf:\n",doc_tfidf[10:20])

doc_bow:
 [(10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]

doc_tfidf:
 [(10, 0.022499206359733896), (11, 0.00729610721246217), (12, 0.006375927830181293), (13, 0.00791776395883734), (14, 0.03010193465712729), (15, 0.013894232750314374), (16, 0.014461760895031426), (17, 0.03010193465712729), (18, 0.00937320863819255), (19, 0.003396739846715514)]


In [7]:
# 根据tf-idf值进行排序
sorted_tfidf = list(sorted(doc_tfidf,key=lambda x:x[1],reverse=True))
top_words = [itos[x[0]] for x in sorted_tfidf[:20]]
print("top_words:",top_words)

top_words: ['christopher', 'ufo', 'colonel', 'aircraft', 'bluejay', 'sergeant', 'photo', 'contribution', 'chronometer', 'breakaway', 'tractor', 'son', 'airman', 'blackjack', 'cygnet', 'fighter', 'geoffrey', 'hapless', 'retrain', 'shaun']


有时候可以在`构建词典时`使用`filter_extremes`方法，进一步对不具备代表性的高频词进行过滤。

**注意**：这个方法对整个字典的影响非常显著，甚至彻底改变top_words的输出。要反复尝试选取最佳参数。

In [8]:
from copy import deepcopy
new_dict = deepcopy(dictionary) # 因为filter_extremes是inplace方法，所以还是复制一份保险
# 排除在所有文档中出现次数不超过5次和出现文档占比超过50%的词
new_dict.filter_extremes(no_below=2,no_above=0.3) 
new_stoi = new_dict.token2id
print("words num:",len(new_stoi))
new_itos = dict(zip(new_stoi.values(),new_stoi.keys()))

new_bow_corpus = [new_dict.doc2bow(doc) for doc in docs]
new_tfidf_model = models.TfidfModel(bow_corpus)

words num: 5439


In [9]:
new_doc_bow = new_dict.doc2bow(docs[0])
new_doc_tfidf = new_tfidf_model[new_doc_bow]

new_sorted_tfidf = list(sorted(new_doc_tfidf,key=lambda x:x[1],reverse=True))
new_top_words = [new_itos[x[0]] for x in new_sorted_tfidf[:20]]
print("top words:",new_top_words)

top words: ['colonel', 'chronometer', 'sun', 'accident', 'son', 'backward', 'gain', 'aircraft', 'sergeant', 'noise', 'achieve', 'crush', 'film', 'passenger', 'black', 'current', 'christopher', 'auxiliary', 'bag', 'gun']


### 小结
我觉得词袋模型并不是一个主模型，而更像是数据预处理的尾巴。通过建立词袋提取关键信息，或是将词袋模型作为接下来的输入。我觉得学到概念就行。